# So'ngi yangiliklar

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd


In [7]:
kunid=1630932185
url = 'https://kun.uz/news/list'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
quotes = soup.find('div', class_="daily-news max-w").find_all(href=True)
links_kun=[]
check_url=[]
for quote in quotes:
    link = "https://kun.uz"+quote['href']
    links_kun.append(link)
    check_url.append(link)

In [8]:
while(True):
    url = 'https://kun.uz/news/list?f=latest&next='+str(kunid)
    kunid=kunid-5000
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    quotes = soup.find('div', class_="daily-news max-w").find_all(href=True)
    for quote in quotes:
        link = "https://kun.uz"+quote['href']
        if link not in check_url:
            if len(check_url)>55:
                check_url.pop(0)
            check_url.append(link)
            links_kun.append(link)
            
    if kunid < 1000000000:
        break
print(len(links_kun))

212471


In [10]:
pd.DataFrame(links_kun).to_csv('links_kun.csv',index=False)       

# Ma'lumotlarni olish

In [ ]:
links_kun = pd.read_csv('links_kun.csv')       

In [ ]:
sikl=0
for i in range(0,links_kun.shape[0]):
    try:
        response_link = requests.get(links_kun.iloc[i,0])
        soup_link = BeautifulSoup(response_link.text, 'lxml')
        quotes_title = soup_link.find('div',class_="single-header__title")  
        quotes_text = soup_link.find('div',class_="single-content")     

        with open(os.path.join('baza_kun','out'+str(sikl)+'.txt'), 'w', encoding="utf-8") as f:
            f.write(quotes_title.text+quotes_text.text)
            f.close()
        sikl=sikl+1
    except Exception as ex:
        pass

In [ ]:
text=''
for i in range(167201):
    f=open("baza_kun/out"+str(i)+".txt","r", encoding="utf-8")
    lines=f.readlines()
    for line in range(len(lines)):
        lines[line]=lines[line].replace("\xa0","")
        if line == 0:
            j=-2
            while True:
                if lines[line][j].isalpha():
                    lines[line]=lines[line][:j+1]+'.\n'
                    break
                j-=1
        if lines[line] != '\n' and lines[line] != '\n\n' and lines[line].find("Фото:") == -1 :
            text+=lines[line]
    if (i+1) % 100 == 0:
        with open('kuns_'+str((i+1)//100)+'.txt', 'w', encoding="utf-8") as ff:
            ff.writelines(text)
            ff.close()
    